# before you go, take a moment to use the best practices for this repository

This code was run with `python 3.9.4`. The packages are listed at the `requirements.txt` file.

I strongly recommend using `pyenv` to maintain multiple versions of python installed locally. 

## install the virtualenv package on your python via pip

at the terminal just run:

`python -m pip install --user virtualenv`

## create your virtual env. 

Once you have your virtual env created, the packages you once install will not affect your local python packages. Which will avoid conflits. A good practice is to have a virtualenv per project.

The community convention is to use the name `venv` for the environment you are about to create. 

to create your virtualven just type the following command at your terminal:

`virtualenv venv` 

and you should see a `/env/` directory inside your project folder or with the given name you choose at the step before.


## step 1: go to your local repository and type on your terminal:

Warning: Now you should active your `virtualenv`. Keep in mind you need to activate everytime you begin to work with the project. Otherwise, the packages you once install on your virtualenv won't be available to your local python.

#### Linux/Macos

Active your virtualenv using the command source, if you are using linux-like bash. If you are using Windows the command can be diffrent. 

`source venv/bin/activate`

If your terminal looks like the above, you are good.

`(venv) dc@blckjzz pegabot-twitter-api % `

#### For Windows Users

`\env\Scripts\activate.bat`


## step 2: once your virtualenv

Now you should have your virtualenv setup. It's time to install the required packages for running this project.

On terminal at the project folder type:

`pip install -r requirements.txt `


## step 3: create your .env file locally

on terminal, inside the project folder just copy and rename the `.env-example` to `.env`

`$ cp .env.example .env`

### Update your virtualenv packages
#### everytime you need to update your requirements.txt file, you should generate an updated version of your `virtualenv` to a requirements.txt file

Go to your terminal and type

`pip freeze > requirements.txt`

After that, you should commit only your `requirements.txt` file instead your virtualenv `/venv` directory (which is ignored by default on this project on .gitignore file)

## Step 4: Using the virtualenv with Jupyter notebooks

For this you should need a package `ipykernel`, which will allow you to link your local venv with your Jupyter notebook. You should do this in order to use the venv and its packages on your notebook. 

At your terminal type:

`source venv/bin/activate` if not activated

`pip install ipykernel` if you have not installed previously. I added this package to the `requirements.txt` file so you should be fine.

`python -m ipykernel install --user --name=${PWD##*/}`

You should see a message like the above:

`Installed kernelspec pegabot-twitter-api in /Users/dc/Library/Jupyter/kernels/pegabot-twitter-api`

Once you hit the command above, you should restart your Jupyter server to load the new Kernel. You may stop the server or just restart the kernel.

Now you have a new kernel for your jupyter notebooks and should indicate which kernel to use in your project.

To do this go to your project, open any file `.ipynb`. At the upper menu bar find `Kernel > Change Kernel > ` and select your new kernel. If you used the same command as above, the kernel should be named as the same as the project, which is `pegabot-twitter-api` in my case.


## Documentação do Tweepy 
`https://docs.tweepy.org/en/stable/`



In [6]:
# load .env variables from local .env file
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [295]:
# if you wish to reload .env variables
%reload_ext dotenv
# %env

In [161]:
# project imports listed bellow #

####################################
import tweepy
import os
import requests
from easydict import EasyDict as edict
####################################

In [66]:
# Loading variables

twitter = {}
twitter['TWITTER_API_KEY'] = %env twitter_api_key 
twitter['TWITTER_API_SECRET'] = %env twitter_api_secret
twitter['TWITTER_API_TOKEN'] = %env twitter_access_token
twitter['TWITTER_API_TOKEN_SECRET'] = %env twitter_access_token_secret


#for key in twitter:
#    print(key, '->', twitter[key])

In [67]:
# Authenticate with Twitter API

twitter['TWITTER_API_TOKEN_SECRET']
auth = tweepy.OAuthHandler(twitter['TWITTER_API_KEY'], twitter['TWITTER_API_SECRET'])

# se não autenticar com o access token algumas funcionalidades da api 
# não ficam disponíveis. tipo verificar rate limits.
auth.set_access_token(twitter['TWITTER_API_TOKEN'], twitter['TWITTER_API_TOKEN_SECRET'])

# o objeto api é utilizado para realizar toda comunicação com a API do twitter.
api = tweepy.API(auth)

In [69]:
# recuperando dados de um twitter user
try:
    user = api.get_user('blckjzz'); 
    print("##############")
    print(user.id_str) # recover user twitter_id
    print(user.name) # prints the name of the user
    print(user.screen_name) # prints the username (@ of the user)
    print(user.followers_count) # 
    print(user.friends_count) # 
    print(user.created_at)
    print(user.verified)
    print("##############")
except tweepy.TweepError as e:
        print("Tweepy Error: {}".format(e))

##############
1080370887357071362
Diego Cerqueira
blckjzz
295
363
2019-01-02 07:51:06
False
##############


In [290]:
# rewrite the json received from the API to a more readable format
def pegabotTransformResponseJson(pa):
    # recovers all data from twitter data key
    lista = {
        # accessing values from profiles
        'username': pa.profiles[0].username,
        'user_id': pa.twitter_data.user_id,
        'user_name': pa.twitter_data.user_name,
        'created_at': pa.twitter_data.created_at,
        'following': pa.twitter_data.following,
        'followers': pa.twitter_data.followers,
        'number_tweets': pa.twitter_data.number_tweets,
        'mentions': pa.twitter_data.mentions,
        # recoverying values for bot probability
        'result_bot_probability': pa.profiles[0].bot_probability.all,
        'result_bot_probability_info': pa.profiles[0].bot_probability.info,
        # recoverying values for sentiment
        'result_sentiment': pa.profiles[0].language_dependent.sentiment.value,
        'result_network': pa.profiles[0].language_independent.network,
        'result_friends': pa.profiles[0].language_independent.friend,
        'result_temporal': pa.profiles[0].language_independent.temporal,
        'result_user': pa.profiles[0].language_independent.user,
        }
    return edict(lista)

In [315]:
def pegabotAPIRequest(profile, env="local"):
    if env == "prod" or env == "production":
        pegabot_api = "https://backend.pegabot.com.br/botometer"
    else:
        pegabot_api = "http://localhost:9000/botometer"
    try:
        params = {'profile': profile, 'search_for':'profile'}
        response = requests.get(pegabot_api, params)    
        jsonResponse = edict(response.json())
        return pegabotTransformResponseJson(jsonResponse)
#        response.raise_for_status()
        # pa.content() # Return the raw bytes of the data payload
        # pa.text() # Return a string representation of the data payload
        # pa.json() # This method is convenient when the API returns JSON
    # Code here will only run if the request is successful
    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err) 

In [318]:
# requests a profile at the pegabot endpoint and transforms it to be readable javascript like in python
profiles = ['blckjzz']
for p in profiles:
    profile = pegabotAPIRequest(p)

blckjzz


In [40]:
# recuperar os X tweets de um perfil
tweets = api.user_timeline(user_id=1080370887357071362, count=100);

for tweet in tweets:
    print("printing the x latests tweets from: " + str(tweet.author.name))
    print(
        str(tweet.id)              + " - " + 
        str(tweet.text)            + " - " + 
        str(tweet.author.name)     + " - " +
        str("É um retweet? [")     + 
            str(tweet.is_quote_status) + 
        str("]")
        )

printing the x latests tweets from: Diego Cerqueira
1436098048296370179 - @yaso @RosieDaSerenata @cuducos @NataliaMazotte @teomoura2 Ótima notícia! Espero que ajude nossos robôs cívicos a n… https://t.co/rmZyRR4sdl - Diego Cerqueira - É um retweet? [False]
printing the x latests tweets from: Diego Cerqueira
1436097861003972608 - Boas novidades para os robôs do bem do Twitter!

@pegabots https://t.co/p5V5WaFAkh - Diego Cerqueira - É um retweet? [True]
printing the x latests tweets from: Diego Cerqueira
1436033000517607427 - RT @CodingRights: Que felicidade ver @ninadhora e @fecampa na Comissão de Transparência das Eleições do TSE. Tarefa fundamental em tempos d… - Diego Cerqueira - É um retweet? [False]
printing the x latests tweets from: Diego Cerqueira
1436031585833390086 - @ninadhora Que incrível, @ninadhora!! Você é muito incrível. Fico feliz em ter conhecido você, inspiração total. Vo… https://t.co/1eE0pyEGNh - Diego Cerqueira - É um retweet? [False]
printing the x latests tweets f

In [ ]:
# engine = parte das análises (apenas) -> jogar um x ou xyx dados e receber um resultado quanto é considerado robô
# API = vou enviar um perfil > vou converter? > dados e jogar no engine <> api <> devolve pra quem pediu
# site (www.pegabot.com.br)
# pegabot lote (analises.pegabot.com.br)
# API do pegabot: https://backend.pegabot.com.br/botometer?profile=blckjzz&search_for=profile

In [56]:
user = api.followers(screen_name="jairbolsonaro", count=20)

In [57]:
for seguidor in user:
    print(seguidor.name + " - " + str(seguidor.created_at))

denerroc_ - 2021-09-10 19:25:34
Bruno Martins - 2020-07-30 19:56:22
Bruno Rheinheimer - 2021-09-10 20:23:31
philippegrego - 2009-05-31 06:06:15
Eduardo Santos - 2021-09-10 20:21:37
JIFER1631305579 - 2021-09-10 20:26:37
victorxx.pvp - 2021-09-10 18:10:21
Notices - 2021-09-10 20:11:47
Ciça Bettencourt 🇧🇷 - 2019-10-30 01:05:02
Denis Alves - 2021-09-10 20:20:04
Eduardo J Machado - 2021-09-08 20:26:05
Samanta Crepaldi - 2021-09-10 20:18:04
João Victor Malandrim - 2021-08-26 10:26:21
Caio Silva - 2021-09-10 20:21:57
SilvaMar - 2021-09-10 20:03:36
Bruna Miranda - 2020-05-23 02:44:35
Liege Therezinha Vogt Barden - 2021-09-10 20:18:22
Marcia Roberta Santos Da Silva - 2021-08-11 22:05:38
Rodrigo Lima - 2021-09-10 20:20:49
Apollo11 - 2021-09-10 20:19:41


In [23]:
# verifica os rate limites da chave em utilização
rateLimits = api.rate_limit_status()
rateLimits['resources']

{'rate_limit_context': {'access_token': '1080370887357071362-WliGiLV7sQ1JZENHzO1ZVCcV4QxRHs'},
 'resources': {'lists': {'/lists/list': {'limit': 15,
    'remaining': 15,
    'reset': 1631219442},
   '/lists/memberships': {'limit': 75, 'remaining': 75, 'reset': 1631219442},
   '/lists/subscribers/show': {'limit': 15,
    'remaining': 15,
    'reset': 1631219442},
   '/lists/members': {'limit': 900, 'remaining': 900, 'reset': 1631219442},
   '/lists/subscriptions': {'limit': 15, 'remaining': 15, 'reset': 1631219442},
   '/lists/show': {'limit': 75, 'remaining': 75, 'reset': 1631219442},
   '/lists/ownerships': {'limit': 15, 'remaining': 15, 'reset': 1631219442},
   '/lists/subscribers': {'limit': 180, 'remaining': 180, 'reset': 1631219442},
   '/lists/members/show': {'limit': 15, 'remaining': 15, 'reset': 1631219442},
   '/lists/statuses': {'limit': 900, 'remaining': 900, 'reset': 1631219442}},
  'application': {'/application/rate_limit_status': {'limit': 180,
    'remaining': 180,
    '

In [62]:
trends = api.trends_available()

trendsBR = api.trends_place(23424768)

# recupera as trends do br com o código
# retorna um json (ver como navegar nele)
# -> trend > name > conteudo

AttributeError: 'dict' object has no attribute 'name'